### Josh Work

In [1]:
import pandas as pd
import json

In [2]:
file_path = '/Users/joshuajpark/Desktop/tc/consulting-spring2025/data/mens/mens_results.csv'
# file_path = '/Users/cjgimena/Desktop/github/consulting-spring2025/data/mens/mens_results.csv'
matches = pd.read_csv(file_path).drop_duplicates()
matches['Date'] = pd.to_datetime(matches['Date'])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/joshuajpark/Desktop/tc/consulting-spring2025/data/mens/mens_results.csv'

In [ ]:
# Helper Functions

def count_sets_won(score_str):
    if not isinstance(score_str, str):
        return 0, 0
    p1_sets = p2_sets = 0
    for s in score_str.split(','):
        s = s.strip()
        parts = s.split('-')
        if len(parts) < 2:
            continue
        try:
            p1 = int(parts[0].split('(')[0])
            p2 = int(parts[1].split('(')[0])
        except:
            continue
        if p1 > p2:
            p1_sets += 1
        else:
            p2_sets += 1
    return p1_sets, p2_sets

def get_winner(row):
    p1_sets, p2_sets = count_sets_won(row['Score'])
    if p1_sets > p2_sets:
        return row['Player1']
    elif p2_sets > p1_sets:
        return row['Player2']
    else:
        return None

if 'Winner' not in matches.columns:
    matches['Winner'] = matches.apply(get_winner, axis=1)

In [ ]:
matches

,Event Name,Date,Player1,Player2,Player1 UTR,Player2 UTR,Score,Winner
0,"Dual Match: University of California, Los Ange...",2025-05-15,Sebastian Eriksson,Spencer Johnson,13.30,13.43,"6-4, 6-4",Sebastian Eriksson
1,"Dual Match: University of California, Los Ange...",2025-05-15,Lucas Brown,Giacomo Revelli,13.00,12.00,"6-2, 5-7",None
2,"Dual Match: University of California, Los Ange...",2025-05-15,Emon Van Loben Sels,Sebastian Gorzny,13.59,13.43,"6-3, 7-6(2)",Emon Van Loben Sels
3,"Dual Match: University of California, Los Ange...",2025-05-15,Pierre Bailly,Kaylan Bigun,13.88,13.24,"6-4, 6-4",Pierre Bailly
4,"Dual Match: University of California, Los Ange...",2025-05-15,Jonah Braswell,Aadarsh Tripathi,13.01,12.91,"7-6(7), 2-1",Jonah Braswell
...,...,...,...,...,...,...,...,...
4899,NaN,2013-09-29,Kaylan Bigun,Saul Baeza De La Paz,13.24,5.00,"6-4, 6-1",Kaylan Bigun
4900,NaN,2013-09-29,Alex Alvarez,Kaylan Bigun,0.00,13.24,"6-2, 6-1",Alex Alvarez
4901,NaN,2013-01-20,Kaylan Bigun,Mateo Moss,13.24,0.00,"6-3, 6-2",Kaylan Bigun
4902,NaN,2013-01-20,Cole Stelse,Kaylan Bigun,12.00,13.24,"6-1, 6-1",Cole Stelse


In [ ]:
def get_player_record(player_name, mens_results_player):
    conf_start = pd.Timestamp('2025-03-06')
    conf_end = pd.Timestamp('2025-04-20')

    conf_matches = mens_results_player[
        (mens_results_player['Date'] >= conf_start) &
        (mens_results_player['Date'] <= conf_end)
    ]

    overall_wins = (mens_results_player['result'] == player_name).sum()
    overall_losses = ((mens_results_player['result'] != player_name) &
                      (mens_results_player['result'] != 'Unfinished')).sum()

    conf_wins = (conf_matches['result'] == player_name).sum()
    conf_losses = ((conf_matches['result'] != player_name) &
                   (conf_matches['result'] != 'Unfinished')).sum()

    return {
        "records": [
            {
                "type": "Overall Record",
                "total": int(overall_wins + overall_losses),
                "won": int(overall_wins),
                "lost": int(overall_losses)
            },
            {
                "type": "Conference Record",
                "total": int(conf_wins + conf_losses),
                "won": int(conf_wins),
                "lost": int(conf_losses)
            }
        ]
    }

In [18]:
record = get_player_record(player_name, mens_results_player)
record

{'records': [{'type': 'Overall Record', 'total': 410, 'won': 293, 'lost': 117},
  {'type': 'Conference Record', 'total': 10, 'won': 7, 'lost': 3}]}

### Cj Work

In [1]:
import pandas as pd
import re
import os

os.getcwd() # Print current working directory

'/Users/cjgimena/Desktop/Github/consulting-spring2025/season_report/summary_page'

In [2]:
os.chdir('../..') # go back two directories
os.getcwd()

'/Users/cjgimena/Desktop/Github/consulting-spring2025'

### Read Data

In [3]:
# USER INPUT: Specify Player!
player_name = 'Rudy Quan'
# player_name = 'Kaylan Bigun'
# # player_name = 'Emon Van Loben Sels'
# player_name = 'Alexander Hoogmartens'
# player_name = 'Spencer Johnson'
# player_name = 'Aadarsh Tripathi'
# player_name = 'Giacomo Revelli'
# player_name = 'Gianluca Ballotta'



In [4]:
combined_data_shots = pd.read_excel(f'data/mens/{player_name}/combined.xlsx', sheet_name='Shots')
combined_data_points = pd.read_excel(f'data/mens/{player_name}/combined.xlsx', sheet_name='Points')
combined_data_sets = pd.read_excel(f'data/mens/{player_name}/combined.xlsx', sheet_name='Sets')

In [41]:
# Subset 2024-2025 Season Matches!
mens_results = pd.read_csv('data/mens/mens_results.csv')[:229]
mens_results['Date'] = pd.to_datetime(mens_results['Date'])


def filter_player(data, player_name):

    # Filter for player_name
    data = data[(data['Player1'] == player_name) | (data['Player2'] == player_name)]

    # Filter for only school events
    data = data[data['Event Name'].str.startswith(('Dual Match', '2024 ITA', '2024-25 NCAA Division'))]
    return data

mens_results_player = filter_player(mens_results, player_name)

In [42]:
mens_results_player

,Event Name,Date,Player1,Player1 UTR,Player2,Player2 UTR,Score
2,"Dual Match: University of California, Los Ange...",2025-04-26,Aidan Kim,13.75,Rudy Quan,13.67,"7-6(3), 6-2"
10,Dual Match: Michigan State University vs Unive...,2025-04-25,Rudy Quan,13.67,Aristotelis THANOS,13.61,"4-6, 3-1"
17,Dual Match: University of Michigan vs Universi...,2025-04-24,Rudy Quan,13.67,William Cooksey,12.00,"6-3, 5-6"
23,"Dual Match: University of California, Los Ange...",2025-04-19,Rudy Quan,13.67,Calvin MUELLER,13.00,"3-6, 6-3, 6-1"
25,"Dual Match: University of California, Los Ange...",2025-04-17,Michael Minasyan,12.00,Rudy Quan,13.67,"2-6, 6-3"
32,Dual Match: Michigan State University vs Unive...,2025-04-12,Rudy Quan,13.67,Ozan Baris,13.83,"6-1, 6-2"
36,"Dual Match: University of California, Los Ange...",2025-04-10,Rudy Quan,13.67,William Cooksey,12.00,"6-3, 6-3"
44,"Dual Match: University of California, Los Ange...",2025-04-05,Rudy Quan,13.67,Charl Morgan,12.00,"6-3, 6-3"
48,Dual Match: Ohio State University vs Universit...,2025-04-03,Alexander Bernard,13.33,Rudy Quan,13.67,"7-6(5), 0-6"
52,"Dual Match: University of California, Los Ange...",2025-04-03,Alexander Bernard,13.33,Rudy Quan,13.67,"7-6(5), 0-6, 3-1"


In [ ]:
# def count_sets_won(score_str):
#     if not isinstance(score_str, str):
#         return 0, 0
#     p1_sets = p2_sets = 0
#     for s in score_str.split(','):
#         s = s.strip()
#         parts = s.split('-')
#         if len(parts) < 2:
#             continue
#         try:
#             p1 = int(parts[0].split('(')[0])
#             p2 = int(parts[1].split('(')[0])
#         except:
#             continue
#         if p1 > p2:
#             p1_sets += 1
#         else:
#             p2_sets += 1
#     return p1_sets, p2_sets

def count_sets_won(score_str):
    if not isinstance(score_str, str):
        return 0, 0
    
    p1_sets = p2_sets = 0
    
    # for s in score_str.split(','):
    #     s = s.strip()
    #     parts = s.split('-')

    sets = [s.strip() for s in score_str.split(',') if s.strip()]  # Clean and split sets
    
    for i, s in enumerate(sets):
        parts = s.split('-')
        
        # Check for valid score format
        if len(parts) < 2:
            continue

        try:
            # Extract the player scores, ignoring tiebreaker details
            p1 = int(parts[0].split('(')[0].strip())
            p2 = int(parts[1].split('(')[0].strip())
        except ValueError:
            continue

        # Check for 10-point tiebreaker format (e.g., "1-0(7)" or "0-1(3)")
        if len(sets) == 3 and i == 2 and "(" in s:
            if (p1 == 1 and p2 == 0) or (p1 == 0 and p2 == 1):
                if p1 == 1:
                    p1_sets += 1
                else:
                    p2_sets += 1
            continue

        # Check for standard set completion
        valid_standard_set = ((p1 >= 6 or p2 >= 6) and abs(p1 - p2) >= 2) or \
                             ((p1 == 7 and p2 == 6) or (p1 == 6 and p2 == 7))

        # If the set is unfinished, skip counting
        if not valid_standard_set:
            continue
        
        # Count the set win for the respective player
        if p1 > p2:
            p1_sets += 1
        else:
            p2_sets += 1
            
    return p1_sets, p2_sets


def get_winner(row):
    p1_sets, p2_sets = count_sets_won(row['Score'])
    if p1_sets == 2:
        return row['Player1']
    elif p2_sets == 2:
        return row['Player2']
    else:
        return 'Unfinished'

In [56]:
mens_results_player[['player1_sets', 'player2_sets']] = mens_results_player['Score'].apply(count_sets_won).apply(pd.Series)
mens_results_player

,Event Name,Date,Player1,Player1 UTR,Player2,Player2 UTR,Score,player1_sets,player2_sets,result
2,"Dual Match: University of California, Los Ange...",2025-04-26,Aidan Kim,13.75,Rudy Quan,13.67,"7-6(3), 6-2",2,0,Aidan Kim
10,Dual Match: Michigan State University vs Unive...,2025-04-25,Rudy Quan,13.67,Aristotelis THANOS,13.61,"4-6, 3-1",0,1,Aristotelis THANOS
17,Dual Match: University of Michigan vs Universi...,2025-04-24,Rudy Quan,13.67,William Cooksey,12.00,"6-3, 5-6",1,0,Rudy Quan
23,"Dual Match: University of California, Los Ange...",2025-04-19,Rudy Quan,13.67,Calvin MUELLER,13.00,"3-6, 6-3, 6-1",2,1,Rudy Quan
25,"Dual Match: University of California, Los Ange...",2025-04-17,Michael Minasyan,12.00,Rudy Quan,13.67,"2-6, 6-3",1,1,Unfinished
32,Dual Match: Michigan State University vs Unive...,2025-04-12,Rudy Quan,13.67,Ozan Baris,13.83,"6-1, 6-2",2,0,Rudy Quan
36,"Dual Match: University of California, Los Ange...",2025-04-10,Rudy Quan,13.67,William Cooksey,12.00,"6-3, 6-3",2,0,Rudy Quan
44,"Dual Match: University of California, Los Ange...",2025-04-05,Rudy Quan,13.67,Charl Morgan,12.00,"6-3, 6-3",2,0,Rudy Quan
48,Dual Match: Ohio State University vs Universit...,2025-04-03,Alexander Bernard,13.33,Rudy Quan,13.67,"7-6(5), 0-6",1,1,Unfinished
52,"Dual Match: University of California, Los Ange...",2025-04-03,Alexander Bernard,13.33,Rudy Quan,13.67,"7-6(5), 0-6, 3-1",1,1,Unfinished


In [57]:
mens_results_player['result'] = mens_results_player.apply(get_winner, axis=1)
mens_results_player

,Event Name,Date,Player1,Player1 UTR,Player2,Player2 UTR,Score,player1_sets,player2_sets,result
2,"Dual Match: University of California, Los Ange...",2025-04-26,Aidan Kim,13.75,Rudy Quan,13.67,"7-6(3), 6-2",2,0,Aidan Kim
10,Dual Match: Michigan State University vs Unive...,2025-04-25,Rudy Quan,13.67,Aristotelis THANOS,13.61,"4-6, 3-1",0,1,Unfinished
17,Dual Match: University of Michigan vs Universi...,2025-04-24,Rudy Quan,13.67,William Cooksey,12.00,"6-3, 5-6",1,0,Unfinished
23,"Dual Match: University of California, Los Ange...",2025-04-19,Rudy Quan,13.67,Calvin MUELLER,13.00,"3-6, 6-3, 6-1",2,1,Rudy Quan
25,"Dual Match: University of California, Los Ange...",2025-04-17,Michael Minasyan,12.00,Rudy Quan,13.67,"2-6, 6-3",1,1,Unfinished
32,Dual Match: Michigan State University vs Unive...,2025-04-12,Rudy Quan,13.67,Ozan Baris,13.83,"6-1, 6-2",2,0,Rudy Quan
36,"Dual Match: University of California, Los Ange...",2025-04-10,Rudy Quan,13.67,William Cooksey,12.00,"6-3, 6-3",2,0,Rudy Quan
44,"Dual Match: University of California, Los Ange...",2025-04-05,Rudy Quan,13.67,Charl Morgan,12.00,"6-3, 6-3",2,0,Rudy Quan
48,Dual Match: Ohio State University vs Universit...,2025-04-03,Alexander Bernard,13.33,Rudy Quan,13.67,"7-6(5), 0-6",1,1,Unfinished
52,"Dual Match: University of California, Los Ange...",2025-04-03,Alexander Bernard,13.33,Rudy Quan,13.67,"7-6(5), 0-6, 3-1",1,1,Unfinished


In [58]:
conf_start = pd.Timestamp('2025-03-06')
conf_end = pd.Timestamp('2025-04-20')

player_conf_matches = mens_results_player[(
                            mens_results_player['Date'] >= conf_start) & 
                            (mens_results_player['Date'] <= conf_end
                                )]

player_conf_matches

,Event Name,Date,Player1,Player1 UTR,Player2,Player2 UTR,Score,player1_sets,player2_sets,result
23,"Dual Match: University of California, Los Ange...",2025-04-19,Rudy Quan,13.67,Calvin MUELLER,13.00,"3-6, 6-3, 6-1",2,1,Rudy Quan
25,"Dual Match: University of California, Los Ange...",2025-04-17,Michael Minasyan,12.00,Rudy Quan,13.67,"2-6, 6-3",1,1,Unfinished
32,Dual Match: Michigan State University vs Unive...,2025-04-12,Rudy Quan,13.67,Ozan Baris,13.83,"6-1, 6-2",2,0,Rudy Quan
36,"Dual Match: University of California, Los Ange...",2025-04-10,Rudy Quan,13.67,William Cooksey,12.00,"6-3, 6-3",2,0,Rudy Quan
44,"Dual Match: University of California, Los Ange...",2025-04-05,Rudy Quan,13.67,Charl Morgan,12.00,"6-3, 6-3",2,0,Rudy Quan
48,Dual Match: Ohio State University vs Universit...,2025-04-03,Alexander Bernard,13.33,Rudy Quan,13.67,"7-6(5), 0-6",1,1,Unfinished
52,"Dual Match: University of California, Los Ange...",2025-04-03,Alexander Bernard,13.33,Rudy Quan,13.67,"7-6(5), 0-6, 3-1",1,1,Unfinished
57,"Dual Match: University of California, Los Ange...",2025-03-31,Makk Peter,13.70,Rudy Quan,13.67,"6-4, 6-4",2,0,Makk Peter
61,Dual Match: University of Washington vs Univer...,2025-03-29,Rudy Quan,13.67,César BOUCHELAGHEM,13.50,"6-4, 4-4",1,0,Unfinished
69,Dual Match: University of Oregon vs University...,2025-03-28,Rudy Quan,13.67,Lenn Luemkemann,12.00,"6-3, 5-3",1,0,Unfinished


In [63]:



overall_wins = (mens_results_player['result'] == player_name).sum()
overall_losses = ((mens_results_player['result'] != player_name) & 
              (mens_results_player['result'] != 'Unfinished')).sum()
conf_wins = (player_conf_matches['result'] == player_name).sum()
conf_losses = ((player_conf_matches['result'] != player_name) & 
              (player_conf_matches['result'] != 'Unfinished')).sum()


pd.DataFrame([{
        'Player': player_name,
        'Overall Record': f"{overall_wins}-{overall_losses}",
        'Conference Record': f"{conf_wins}-{conf_losses}"
    }])

,Player,Overall Record,Conference Record
0,Rudy Quan,13-6,6-1
